In [1]:
import mpramnist
from mpramnist.Fluorescence.dataset import FluorescenceDataset

from mpramnist.models import HumanLegNet
from mpramnist.models import initialize_weights
from mpramnist.trainers import LitModel_AgarwalJoint

from mpramnist import transforms as t
from mpramnist import target_transforms as t_t

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pytorch_lightning as L

# Regression task

In [2]:
BATCH_SIZE = 32
NUM_WORKERS = 103

In [3]:
train_transform = t.Compose([
    t.Seq2Tensor(),
    t.ReverseComplement(0.5)
])
val_test_transform = t.Compose([
    t.Seq2Tensor()
])

In [4]:
task = "regression"
activity_columns = ["JURKAT", "K562", "THP1"]

train_dataset = FluorescenceDataset(task = task, activity_columns = activity_columns, split = "train",transform = train_transform)

val_dataset = FluorescenceDataset(task = task, activity_columns = activity_columns, split = "val",transform = val_test_transform) 

test_dataset = FluorescenceDataset(task = task, activity_columns = activity_columns, split = "test", transform = val_test_transform)

In [5]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset FluorescenceDataset of size 12335 (MpraDaraset)
    Number of datapoints: 12335
    Used split fold: train
Dataset FluorescenceDataset of size 3353 (MpraDaraset)
    Number of datapoints: 3353
    Used split fold: test


In [6]:
# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = NUM_WORKERS)

val_loader = data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS)

test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS)

In [7]:
in_channels = len(train_dataset[0][0])
out_channels = len(activity_columns)

In [8]:
model = HumanLegNet(in_ch=in_channels,
                     output_dim = out_channels,
                     stem_ch=64,
                     stem_ks=11,
                     ef_ks=9,
                     ef_block_sizes=[80, 96, 112, 128],
                     pool_sizes=[2,2,2,2],
                     resize_factor=4)
model.apply(initialize_weights)

seq_model = LitModel_AgarwalJoint(model = model, num_outputs = out_channels,
                           loss = nn.MSELoss(),
                           weight_decay = 1e-1, lr = 1e-2, print_each = 1)

In [9]:
# Initialize a trainer
trainer = L.Trainer(
    accelerator="gpu",
    devices=[0],
    max_epochs=2,
    gradient_clip_val=1,
    precision='16-mixed', 
    enable_progress_bar = True,
    num_sanity_val_steps=0
)

# Train the model
trainer.fit(seq_model,
            train_dataloaders = train_loader,
            val_dataloaders = val_loader)
trainer.test(seq_model, dataloaders = test_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-04-10 20:41:28.055305: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 20:41:28.070438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one 

Training: |                                                                                       | 0/? [00:00…

Validation: |                                                                                     | 0/? [00:00…


-------------------------------------------------------------------------------
| Epoch: 0 | Val Loss: 1.04345 | Val Pearson: 0.41669 | Train Pearson: 0.30089 
-------------------------------------------------------------------------------



Validation: |                                                                                     | 0/? [00:00…

...


-------------------------------------------------------------------------------
| Epoch: 1 | Val Loss: 0.91191 | Val Pearson: 0.51501 | Train Pearson: 0.46526 
-------------------------------------------------------------------------------



LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |                                                                                        | 0/? [00:00…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.8785858750343323     │
│       test_pearson        │    0.5315124988555908     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.8785858750343323, 'test_pearson': 0.5315124988555908}]

# Classification task

TODO LitModel_Fluorescence_classification

In [12]:
BATCH_SIZE = 32
NUM_WORKERS = 103
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
train_transform = t.Compose([
    t.Seq2Tensor()
])
val_test_transform = t.Compose([
    t.Seq2Tensor()
])
task = "classification"
activity_columns = ["JURKAT", "K562", "THP1"]
train_dataset = FluorescenceDataset(task = task, activity_columns = activity_columns, split = "train",transform = train_transform)

val_dataset = FluorescenceDataset(task = task, activity_columns = activity_columns, split = "val",transform = val_test_transform) 

test_dataset = FluorescenceDataset(task = task, activity_columns = activity_columns, split = "test", transform = val_test_transform)

cuda:0


In [13]:
in_channels = len(train_dataset[0][0])
out_channels = len(activity_columns)

In [14]:
import pytorch_lightning as L
from torch.nn import functional as F
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
    
class Seq1Model(L.LightningModule):
    
    def __init__(self, in_ch, out_ch, lr=3e-4):
        super().__init__()
        
        self.model = HumanLegNet(in_ch=in_ch,
                                 output_dim = out_ch,
                                 stem_ch=64,
                                 stem_ks=11,
                                 ef_ks=9,
                                 ef_block_sizes=[80, 96, 112, 128],
                                 pool_sizes=[2,2,2,2],
                                 resize_factor=4)
        self.model.apply(initialize_weights)
        

        self.loss = torch.nn.CrossEntropyLoss().to(device)
        self.lr = lr
        self.n_classes = out_ch

        self.val_loss = torch.tensor([], device=device)
        self.y_score = torch.tensor([], device=device)
        self.y_true = torch.tensor([], device=device)
        self.train_loss = torch.tensor([], device=device)

        
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_nb):
        X, y = batch
        y_hat = self.model(X)
        y = y.long()

        loss = self.loss(y_hat, y)
        
        self.log("train_loss", loss, prog_bar=True,  on_step=True, on_epoch=True, logger = True)
        self.train_loss = torch.cat([self.train_loss, loss.unsqueeze(0)])
        
        lr = self.optimizers().param_groups[0]['lr']  # Get current learning rate
        self.log('learning_rate', lr, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        y = y.long()

        loss = self.loss(y_hat, y)
        
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)

        self.val_loss = torch.cat([self.val_loss, loss.unsqueeze(0)])
        self.y_score = torch.cat([self.y_score, y_hat])
        self.y_true = torch.cat([self.y_true, y])
        
    def on_validation_epoch_end(self):
        
        val_loss = self.val_loss.mean()
        
        auroc = self.calculate_auroc(y_score=self.y_score, y_true=self.y_true, n_classes=self.n_classes) 
        arr, precision, recall, accuracy, f1, aupr = self.calculate_aupr(y_score=self.y_score, y_true=self.y_true, n_classes=self.n_classes) 
        
        res_str = '|' + ' {}: {:.5f} |'.format("Current_epoch", self.current_epoch) 
        res_str += ' {}: {:.5f} |'.format("Val_loss", val_loss)
        res_str += ' {}: {:.5f} |'.format("Val_AUCROC", auroc)
        res_str += ' {}: {:.5f} |'.format("Val_AUPR", aupr)
        res_str += "\n"
        res_str += "|"
        res_str += ' {}: {} |'.format("Number of predicted values", arr)
        res_str += "\n"
        res_str += "|"
        res_str += ' {}: {:.5f} |'.format("Precision", precision)
        res_str += ' {}: {:.5f} |'.format("Recall", recall)
        res_str += ' {}: {:.5f} |'.format("Accuracy", accuracy)
        res_str += ' {}: {:.5f} |'.format("F1", f1)
        if len(self.train_loss) != 0: 
            train_loss = self.train_loss.mean()
            res_str += ' {}: {:.5f} |'.format("Train_loss", train_loss)
            self.train_loss = torch.tensor([], device=device)
        
        border = '-'*100
        print("\n".join(['',border, res_str, border,'']))
        
        self.val_loss = torch.tensor([], device=device)
        self.y_score = torch.tensor([], device=device)
        self.y_true = torch.tensor([], device=device)

        return None

    def test_step(self, batch, _):
        x, y = batch
        y_hat = self.model(x)
        y = y.long()

        loss = self.loss(y_hat, y)
        self.log('test_loss', 
                 loss, 
                 prog_bar=True, 
                 on_step=False,
                 on_epoch=True)

    def calculate_auroc(self, y_score, y_true, n_classes):
        y_score = F.softmax(y_score.float(), dim=1).cpu().numpy()
        y_true = y_true.cpu().numpy()

        y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))
        
        return roc_auc_score(y_true_bin, y_score, multi_class="ovr", average="macro")

    def calculate_aupr(self, y_score, y_true, n_classes):
        y_score = F.softmax(y_score.float(), dim=1).cpu().numpy()
        y_pred = np.argmax(y_score, axis=1)
        y_true = y_true.cpu().numpy()
        
        arr = []
        for i in range(n_classes):
            arr.append(y_pred.tolist().count(i))
        
        precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_true, y_pred, average='macro', zero_division = 0)  
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')

        y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))
        pr_auc = average_precision_score(y_true_bin, y_score, average="macro")
        return arr, precision, recall, accuracy, f1, pr_auc

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.model(x)          
        
        return {"y": y.squeeze().long().cpu().detach().float(), "pred": y_hat.cpu().detach().float()}

    def train_dataloader(self):
        return data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = NUM_WORKERS, pin_memory=True)

    def val_dataloader(self):
        return data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS, pin_memory=True)
    
    def test_dataloader(self):
        return data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS, pin_memory=True)

    def configure_optimizers(self):
        
        self.optimizer = torch.optim.AdamW(self.parameters(),
                                               lr=self.lr,
                                               weight_decay = 1e-2)
        
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, # type: ignore
                                                        max_lr=self.lr,
                                                        three_phase=False, 
                                                        total_steps=self.trainer.estimated_stepping_batches, # type: ignore
                                                        pct_start=0.3,
                                                        cycle_momentum =False)
        lr_scheduler_config = {
                    "scheduler": lr_scheduler,
                    "interval": "step",
                    "frequency": 1,
                    "name": "cycle_lr"
            }
            
        return [self.optimizer], [lr_scheduler_config]


In [15]:
seq_model = Seq1Model(in_ch = in_channels, out_ch = out_channels, lr = 1e-2)

# Initialize a trainer
trainer = L.Trainer(
    accelerator="gpu",
    devices=[0],
    max_epochs=5,
    gradient_clip_val=1,
    precision='16-mixed', 
    enable_progress_bar = True,
    num_sanity_val_steps=0
)

# Train the model
trainer.fit(seq_model)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-03-31 22:30:28.219202: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 22:30:28.253744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one 

Training: |                                                                                                   …

RuntimeError: Expected floating point type for target with class probabilities, got Long